<a href="https://colab.research.google.com/github//asabenhur/CS425/blob/master/notebooks/04_global_alignment.ipynb">
  <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Global Alignment:  The Needleman Wunsch Algorithm

The objective of this notebook is to help you familiarize yourself with the Needleman Wunsch algorithm for pairwise alignment of sequences.

In [1]:
import numpy as np

In [2]:
# to print colored arrows you will need the termcolor module
# if you don't have it, traceback arrows will be printed 
# without color
color = True
try :
    from termcolor import colored
except :
    color = False

# the three directions you can go in the traceback:
DIAG = 0 
UP = 1 
LEFT = 2
# UTF-8 representations of arrow symbols
# arrows[DIAG] is a diagonal arrow
# arrows[UP] is an up arrow
# arrows[LEFT] is a left
arrows = [u"\u2196", u"\u2191", u"\u2190"]

def needleman_wunsch_matrix(seq1, seq2, 
                            match=1, mismatch=-1, indel=-1):
    """
    Fill the DP matrix according to the Needleman-Wunsch 
    algorithm for two sequences seq1 and seq2.
    match:  the match score
    mismatch:  the mismatch score
    indel:  the indel score
    
    Returns the matrix of scores and the matrix of pointers
    """
    n = len(seq1)
    m = len(seq2)
    s = np.zeros( (n+1, m+1) ) # DP matrix
    ptr = np.zeros( (n+1, m+1), dtype=int  ) # matrix of pointers

    ##### INITIALIZE SCORING MATRIX (base case) #####

    for i in range(1, n+1) :
        s[i,0] = indel * i
    for j in range(1, m+1):
        s[0,j] = indel * j
        
    ########## INITIALIZE TRACEBACK MATRIX ##########

    # Tag first row by LEFT, indicating initial '-'s
    ptr[0,1:] = LEFT
        
    # Tag first column by UP, indicating initial '-'s
    ptr[1:,0] = UP

    #####################################################

    for i in range(1,n+1):
        for j in range(1,m+1): 
            # match
            if seq1[i-1] == seq2[j-1]:
                s[i,j] = s[i-1,j-1] + match
                ptr[i,j] = DIAG
            # mismatch
            else :
                s[i,j] = s[i-1,j-1] + mismatch
                ptr[i,j] = DIAG
            # indel penalty
            if s[i-1,j] + indel > s[i,j] :
                s[i,j] = s[i-1,j] + indel
                ptr[i,j] = UP
            # indel penalty
            if s[i, j-1] + indel > s[i,j]:
                s[i,j] = s[i, j-1] + indel
                ptr[i,j] = LEFT

    return s, ptr

def needleman_wunsch_trace(seq1, seq2, s, ptr) :

    #### TRACE BEST PATH TO GET ALIGNMENT ####
    align1 = ""
    align2 = ""
    n, m = (len(seq1), len(seq2))
    i = n
    j = m
    curr = ptr[i, j]
    while (i > 0 or j > 0):        
        ptr[i,j] += 3
        if curr == DIAG :            
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1            
        elif curr == LEFT:
            align1 = '-' + align1
            align2 = seq2[j-1] + align2
            j -= 1            
        elif curr == UP:
            align1 = seq1[i-1] + align1
            align2 = '-' + align2
            i -= 1
           
        curr = ptr[i,j]

    return align1, align2


In [3]:
def show_ptr_matrix(ptr, seq1, seq2) :

    print('\n'+'~`'*25)
    print("Traceback")
    global color
    print(" " + " ".join(seq2))
    for i in range(len(ptr)) :
        if (i > 0) :
            print (seq1[i-1] + " ",end="")
        if (i == 0) :
            print("  ",end="")
        for j in range(len(ptr[i])) :
            if color and ptr[i,j] >= 3 :
                print(" " + colored(arrows[ptr[i,j]-3], 'green' ),
                      end="")
            else :
                if ptr[i,j] >=3 :
                    ptr[i,j] -=3
                print(" " + arrows[ptr[i,j]],end="")
        print() 

def show_dp_matrix(s, seq1, seq2) :

    print('\n'+'~`'*25)
    print("DP matrix")
    print("         " + "    ".join(seq2))
    for i in range(len(s)) :
        if (i > 0) :
            print(seq1[i-1] + " ",end="")
        if (i == 0) :
            print("  ",end="")
        for j in range(len(s[i])) :
            print(" " + "% 2.1f" % s[i,j],end="")
        print() 
    

In [4]:
def needleman_wunsch(seq1, seq2, match=1, mismatch=-1, indel=-1,
                     verbose=True) :
    """
    computes an optimal global alignment of two sequences using 
    the Needleman-Wunsch algorithm
    returns the alignment and its score
    """
    s,ptr = needleman_wunsch_matrix(seq1, seq2, 
                                    match, mismatch, indel)
    alignment = needleman_wunsch_trace(seq1, seq2, s, ptr)

    if verbose :
        show_dp_matrix(s, seq1, seq2)
        show_ptr_matrix(ptr, seq1, seq2)
        print('\n'+'~`'*25)
        print("Alignment Score: %f\n" % (s[len(seq1),len(seq2)]))
        print("Alignment:")
        print(alignment[0])
        print(alignment[1])
    
    return alignment, s[len(seq1), len(seq2)]


In [5]:
from random import randint
def random_DNA_sequence(length):
    """
    Returns a random DNA of the given length.
    """
    nucleotides = ['A','T','G','C']
    seq = [ nucleotides[randint(0,3)] for i in range(length) ]
    return ''.join(seq)

In [6]:
seq1 = random_DNA_sequence(10)
seq2 = random_DNA_sequence(10)

In [7]:
needleman_wunsch(seq1, seq2, 1, -1, -1)


~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`
DP matrix
         A    T    G    T    C    G    C    T    T    A
    0.0 -1.0 -2.0 -3.0 -4.0 -5.0 -6.0 -7.0 -8.0 -9.0 -10.0
A  -1.0  1.0  0.0 -1.0 -2.0 -3.0 -4.0 -5.0 -6.0 -7.0 -8.0
T  -2.0  0.0  2.0  1.0  0.0 -1.0 -2.0 -3.0 -4.0 -5.0 -6.0
A  -3.0 -1.0  1.0  1.0  0.0 -1.0 -2.0 -3.0 -4.0 -5.0 -4.0
C  -4.0 -2.0  0.0  0.0  0.0  1.0  0.0 -1.0 -2.0 -3.0 -4.0
A  -5.0 -3.0 -1.0 -1.0 -1.0  0.0  0.0 -1.0 -2.0 -3.0 -2.0
C  -6.0 -4.0 -2.0 -2.0 -2.0  0.0 -1.0  1.0  0.0 -1.0 -2.0
T  -7.0 -5.0 -3.0 -3.0 -1.0 -1.0 -1.0  0.0  2.0  1.0  0.0
C  -8.0 -6.0 -4.0 -4.0 -2.0  0.0 -1.0  0.0  1.0  1.0  0.0
C  -9.0 -7.0 -5.0 -5.0 -3.0 -1.0 -1.0  0.0  0.0  0.0  0.0
G  -10.0 -8.0 -6.0 -4.0 -4.0 -2.0  0.0 -1.0 -1.0 -1.0 -1.0

~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`
Traceback
 A T G T C G C T T A
   ↖ ← ← ← ← ← ← ← ← ← ←
A  ↑ ↖ ← ← ← ← ← ← ← ← ↖
T  ↑ ↑ ↖ ← ↖ ← ← ← ↖ ↖ ←
A  ↑ ↖ ↑ ↖ ↖ ↖ ↖ ↖ ↖ ↖ ↖
C  ↑ ↑ ↑ ↖ ↖ ↖ ← ↖ ← ← ←
A  ↑ ↖ ↑ ↖ ↖ ↑ ↖ ↖ ↖ ↖ ↖
C  ↑ ↑ ↑ 

(('AT-ACACTCCG', 'ATGTCGCT-TA'), -1.0)

In [8]:
needleman_wunsch(seq1, seq2, 1, -1, -0.1)


~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`
DP matrix
         A    T    G    T    C    G    C    T    T    A
    0.0 -0.1 -0.2 -0.3 -0.4 -0.5 -0.6 -0.7 -0.8 -0.9 -1.0
A  -0.1  1.0  0.9  0.8  0.7  0.6  0.5  0.4  0.3  0.2  0.1
T  -0.2  0.9  2.0  1.9  1.8  1.7  1.6  1.5  1.4  1.3  1.2
A  -0.3  0.8  1.9  1.8  1.7  1.6  1.5  1.4  1.3  1.2  2.3
C  -0.4  0.7  1.8  1.7  1.6  2.7  2.6  2.5  2.4  2.3  2.2
A  -0.5  0.6  1.7  1.6  1.5  2.6  2.5  2.4  2.3  2.2  3.3
C  -0.6  0.5  1.6  1.5  1.4  2.5  2.4  3.5  3.4  3.3  3.2
T  -0.7  0.4  1.5  1.4  2.5  2.4  2.3  3.4  4.5  4.4  4.3
C  -0.8  0.3  1.4  1.3  2.4  3.5  3.4  3.3  4.4  4.3  4.2
C  -0.9  0.2  1.3  1.2  2.3  3.4  3.3  4.4  4.3  4.2  4.1
G  -1.0  0.1  1.2  2.3  2.2  3.3  4.4  4.3  4.2  4.1  4.0

~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`~`
Traceback
 A T G T C G C T T A
   ↖ ← ← ← ← ← ← ← ← ← ←
A  ↑ ↖ ← ← ← ← ← ← ← ← ←
T  ↑ ↑ ↖ ← ↖ ← ← ← ↖ ↖ ←
A  ↑ ↖ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↖
C  ↑ ↑ ↑ ↑ ↑ ↖ ← ↖ ← ← ↑
A  ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↖
C  ↑ ↑ ↑ ↑ 

(('A--TAC-AC-T-CCG', 'ATGT-CG-CTTA---'), 4.000000000000002)